# Basic CNN Pipeline

### 1. Load and preprocess data

In [ ]:
import torch
import numpy as np

from helper_functions.encode import (
    prepare_training_data, CHARACTERS, 
)

train_folder = "../data/train/combine"
test_folder = "../data/test/combine"

print("Preparing training data...")
X_train, _, y_train_onehot, _, _ = prepare_training_data(train_folder)
y_train = torch.tensor(np.argmax(y_train_onehot, axis=1), dtype=torch.long)

print("Preparing test data...")
X_test, _, y_test_onehot, _, _ = prepare_training_data(test_folder)
y_test = torch.tensor(np.argmax(y_test_onehot, axis=1), dtype=torch.long)

### 2. Instantiate basic CNN model and train it

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from helper_functions.models.basic_cnn import CharDataset, CNN


# Create dataloaders
train_loader = DataLoader(CharDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(CharDataset(X_test, y_test), batch_size=32, shuffle=False)

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(num_classes=len(CHARACTERS)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
print("Training model...")
for epoch in range(15):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
    
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    print(f"Epoch {epoch+1} Loss: {running_loss/len(train_loader.dataset):.4f}")


In [ ]:
from helper_functions.models.basic_cnn import evaluate_character_level, evaluate_captcha_level

"""
3. Evaluate basic CNN model
"""
print("\nCharacter-Level Evaluation:")
char_metrics = evaluate_character_level(model, test_loader, device)
for metric, value in char_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f}")

print("\nCaptcha-Level Evaluation:")
captcha_acc = evaluate_captcha_level(model, test_folder, device)
print(f"Accuracy: {captcha_acc:.4f}")